# LLM 파인튜닝

### 1. 모델 로드

In [ ]:
# 양자화 결정(data 사이즈나 상황에 따라)
# parameter: quantization_config=BitsAndBytesConfig()

### 2. 토크나이저 불러오기

In [1]:
# 텍스트 -> 텐서

### 3. 학습 데이터 준비

#### 1) Huggingface 제공 데이터셋 불러오기

#### 2) 커스텀 데이터셋 만들기

#### 3) 학습 데이터로 만들기

### 4. 학습

#### 1) 학습 설정

#### 2) 학습

#### 3) 학습된 모델을 저장